## Debugging Commands for Ease of Access (Run Last Block before Using)

#### Delete Cluster & IAM Role

In [ ]:
deleteRedshiftCluster()

#### Check Cluster Status

In [ ]:
checkClusterStatus()

#### Check DWH Parameters 

In [ ]:
checkDWHParm()

#### Check DWH Information

In [ ]:
checkDWHInfo()

#### Insert Tables

In [ ]:
insert_tables(cur, conn)

## Test Queries to Ensure that Data has been Inserted Into Tables

#### Check if Data Exists in --- [staging_events]

In [ ]:
%sql $check_staging_events

#### Check if Data Exists in --- [staging_songs] 

In [ ]:
%sql $check_staging_songs

#### Check if Data Exists in --- [songplay_table] 

In [ ]:
%sql $check_songplay_table

#### Check if Data Exists in --- [user_table]

In [ ]:
%sql $check_user_table

#### Check if Data Exists in --- [song_table]

In [ ]:
%sql $check_song_table

#### Check if Data Exists in --- [artist_table]

In [ ]:
%sql $check_artist_table

#### Check if Data Exists in --- [time_table]

In [ ]:
%sql $check_time_table

## NOTE: RUN BELOW BEFORE RUNNING ANY OF THE COMMANDS ABOVE

In [ ]:
import configparser
import psycopg2
import boto3
import json
import pandas as pd
import time 
from sql_queries import copy_table_queries, insert_table_queries

global config
config = configparser.ConfigParser()
config.read('dwh.cfg')

def deleteRedshiftCluster():
    redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True)
    iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
    
    checkClusterStatus()

def checkClusterStatus():
    status = False
    waiting = False

    while status != True:
        try: 
            myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, TagKeys=['ClusterStatus'])['Clusters'][0]
            clusterStatus = myClusterProps['ClusterStatus']
            if clusterStatus == 'available':
                print ("CLUSTER STATUS: Cluster is available")
                status = True
                waiting = False
            elif ((clusterStatus == 'creating') and (waiting == False)):
                print ("CLUSTER STATUS: Cluster is being created...")
                status = False
                waiting = True
            elif ((clusterStatus == 'deleting') and (waiting == False)):
                print ("CLUSTER STATUS: Cluster is being deleted...")
                status = False
                waiting = True
            elif waiting == True:
                print("...")
                time.sleep(10)
        except:
            status = True
            print("CLUSTER STATUS: There is no active Redshift cluster.")


KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")
    
(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)
    
s3 = boto3.resource('s3', region_name = "us-west-2", aws_access_key_id = KEY, aws_secret_access_key = SECRET)
iam = boto3.client('iam', region_name = "us-west-2", aws_access_key_id = KEY, aws_secret_access_key = SECRET)
redshift = boto3.client('redshift', region_name = "us-west-2", aws_access_key_id = KEY, aws_secret_access_key = SECRET)

def checkDWHParm():
    return pd.DataFrame({"Param":
                      ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
                  "Value":
                      [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
                 })

def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

def checkDWHInfo():
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    return prettyRedshiftProps(myClusterProps)

def insert_tables(cur, conn):
    for query in insert_table_queries:
        cur.execute(query)
        print("Preparing to Insert: {}".format(query))
        conn.commit()
        
        print("Table has been inserted")

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

DWH_ENDPOINT="dwhcluster.cjaudyn1b0ap.us-west-2.redshift.amazonaws.com"
DWH_ROLE_ARN="arn:aws:iam::114410147566:role/awsprojectuser"
    
# ==================
import os 
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)



%load_ext sql
%sql $conn_string

check_staging_events = '''
    SELECT * FROM staging_events
    LIMIT 5
'''

check_staging_songs = '''
    SELECT * FROM staging_songs
    LIMIT 5
'''

check_songplay_table = '''
    SELECT * FROM songplay
    LIMIT 5
'''

check_user_table = '''
    SELECT * FROM users
    LIMIT 5
'''

check_song_table = '''
    SELECT * FROM songs
    LIMIT 5
'''

check_artist_table = '''
    SELECT * FROM artist
    LIMIT 5
'''

check_time_table = '''
    SELECT * FROM time
    LIMIT 5
'''
    